<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220518_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
# 할당된 장치 보기
tf.debugging.set_log_device_placement(True)

a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
b = tf.constant([[1., 2.], [3., 4.], [5. ,6.]])
c = tf.matmul(a, b)

print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [ ]:
# 수동으로 장치 할당하기
tf.debugging.set_log_device_placement(True)

with tf.device('/CPU:0'):
  a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
  b = tf.constant([[1., 2.], [3., 4.], [5. ,6.]])

c = tf.matmul(a, b) # 수행할 장치 명시적인 할당 X - 가용한 장치들 중 하나를 고르고, 필요하다면 텐서를 자동으로 복사함

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


## GPU 메모리 제한
`tf.config.experimental.set_visible_devices` : 텐서플로우에서 접근할 수 있는 GPU 조정 가능

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    print(e)

- 가용한 메모리 일부에만 할당되도록 하거나, 프로세스 요구량만큼 메모리 사용이 가능할 필요가 있다.
1. `tf.config.experimental.set_memory_growth` : 메모리 증가 허용 - 런타임에서 할당하는데 필요한 양만큼의 gpu 메모리를 할당함.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 에러 : 프로그램 시작 시에 메모리 증가가 설정되어야 함
    print(e)

Physical devices cannot be modified after being initialized


2. `tf.config.experimental.set_virtual_device_configuration`으로 가상 GPU 장치를 설정, GPU에 할당될 전체 메모리를 제한함

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  # 1번째 gpu에 1gb 메모리만 할당함
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 1024)]
    )
  except RuntimeError as e:
    # 얘도 에러는 똑같이 뜸 - 이미 초기화를 했기 때문
    print(e)

Virtual devices cannot be modified after being initialized


## 멀티 GPU 시스템에서 하나의 GPU만 사용하기
- 2개 이상의 GPU가 있다면 기본적으로 낮은 GPU가 선택됨. 다른 GPU에서 실행하고 싶다면 명시적인 표시가 필요함

In [ ]:
tf.debugging.set_log_device_placement(True)

try:
  with tf.device('/device:GPU:2'):
    a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
    b = tf.constant([[1., 2.], [3., 4.], [5., 6.]])
    c = tf.matmul(a, b)
except RuntimeError as e:
  print(e)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
# 명시한 장치가 없을 때 텐서플로우가 자동으로 현재 지원하는 장치를 선택하게 함 : tf.config.set_soft_device_placement(True)
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
b = tf.constant([[1., 2.], [3., 4.], [5., 6.]])
c = tf.matmul(a, b)

print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


## 멀티 GPU 사용하기 : `tf.distribute.Strategy`
- `MirroredStrategy` : 입력 데이터를 나누고 모델의 각 복사본을 각 GPU에서 실행함. `데이터 병렬처리` 라고도 함.

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  inputs = tf.keras.layers.Input(shape = (1, ))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs = inputs, outputs = predictions)
  model.compile(loss = 'mse',
                optimizer = tf.keras.optimizers.SGD(learning_rate = 0.2))

### `tf.distribute.strategy` 미사용
- `tf.distribute.Strategy`는 여러 장치에 걸쳐 계산을 복제해서 동작함. 모델을 각 GPU에 구성해서 수동으로 이를 구현할 수도 있다.

In [ ]:
tf.debugging.set_log_device_placement(True)

gpus = tf.config.experimental.list_logical_devices('GPU')
if gpus:
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
      a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
      b = tf.constant([[1., 2.], [3., 4.], [5., 6.]])
      c.append(tf.matmul(a, b))

  with tf.device('/CPU:0'):
    matmul_sum = tf.add_n(c)

  print(matmul_sum)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
